In [1]:
import logging
import numpy as np
import pandas as pd

from validata.parser import Parser
from validata.validator import Validator

from validata.operators import Operator
from validata.comparators import Comparator

%load_ext autoreload
%autoreload 2

In [2]:
# Enable to see what is going on under the hood
# logging.basicConfig(level=logging.DEBUG)

## Create some data

In [3]:
households = pd.DataFrame({
    "id": np.arange(10),
    "size": [1, 1, 1, 1, 2, 2, 2, 2, 3, 4],
    "income_1": [35e3, 55e3, 110e3, None, 40e3, 42e3, 73e3, 38e3, None, 57e3],
    "income_2": [None, None,  None, None, 35e3, 22e3, None, 30e3, None, 45e3],
})

households

,id,size,income_1,income_2
0,0,1,35000.0,NaN
1,1,1,55000.0,NaN
2,2,1,110000.0,NaN
3,3,1,NaN,NaN
4,4,2,40000.0,35000.0
5,5,2,42000.0,22000.0
6,6,2,73000.0,NaN
7,7,2,38000.0,30000.0
8,8,3,NaN,NaN
9,9,4,57000.0,45000.0


## Performing a single validation

Validata lets you perform validations on each record in the data set. In the simplest form, a validation is formulated as a single logical condition, for example:

```
income_1 < 50000
```

In this example, `income_1` is a column in a data set. The `<` sign is a comparison operator (aka Comparator) and `50000` the value to compare against. The general form for a validation thus is:

```
<column name> <comparison operator> <value>
```

The `Parser` class allows you to perform a single validation, as is demonstrated below:

In [4]:
# Create a Parser and supply it with a validation
ps = Parser("income_1 < 50000", "check_income_below_50K")

# Perform the validation on the households data set
households.join(ps.evaluate(households))

,id,size,income_1,income_2,check_income_below_50K
0,0,1,35000.0,NaN,True
1,1,1,55000.0,NaN,False
2,2,1,110000.0,NaN,False
3,3,1,NaN,NaN,False
4,4,2,40000.0,35000.0,True
5,5,2,42000.0,22000.0,True
6,6,2,73000.0,NaN,False
7,7,2,38000.0,30000.0,True
8,8,3,NaN,NaN,False
9,9,4,57000.0,45000.0,False


As you can see, the `Parser` applied the logical validation to each row of the `households` data set. It returned `True` for all rows with `income_1` smaller than `30000` and `False` otherwise.

To see which other types of comparisons are available, use the `list()` method of the `Comparator` base class like so:

In [5]:
# Show which comparison operators are available
Comparator.list()

{'!=', '<', '<=', '==', '>', '>=', 'between', 'in', 'missing', 'not missing'}

### Combining Validations

The `Parser` also allows you to combine validation checks using `and` or `or`. When `and` is used both the left hand and the right hand condition need to be `True` for the entire validation to be `True`. With `or` only one condition needs to be `True`. Some simple examples:

In [6]:
# Both income columns larger than 50.000
both_incomes_high = "income_1 > 50000 and income_2 > 50000"

# Either income is above 50.000
one_income_high = "income_1 > 50000 or income_2 > 50000"

# Feel free to test using the Parser
ps = Parser(one_income_high, "check_one_income_above_50K")
households.join(ps.evaluate(households))

,id,size,income_1,income_2,check_one_income_above_50K
0,0,1,35000.0,NaN,False
1,1,1,55000.0,NaN,True
2,2,1,110000.0,NaN,True
3,3,1,NaN,NaN,False
4,4,2,40000.0,35000.0,False
5,5,2,42000.0,22000.0,False
6,6,2,73000.0,NaN,True
7,7,2,38000.0,30000.0,False
8,8,3,NaN,NaN,False
9,9,4,57000.0,45000.0,True


In addition, you can group conditions using brackets `(...)`. Especially when using `and` and `or` together, this prevents ambiguous statements. For example: 

In [7]:
# Cases where an income may be missing
extra_income = """
    (size == 1 and income_1 missing and income_2 missing) or
    (size >= 2 and (income_1 missing or income_2 missing)
"""

ps = Parser(extra_income, "maybe_missing_income")
households.join(ps.evaluate(households))

,id,size,income_1,income_2,maybe_missing_income
0,0,1,35000.0,NaN,False
1,1,1,55000.0,NaN,False
2,2,1,110000.0,NaN,False
3,3,1,NaN,NaN,True
4,4,2,40000.0,35000.0,False
5,5,2,42000.0,22000.0,False
6,6,2,73000.0,NaN,True
7,7,2,38000.0,30000.0,False
8,8,3,NaN,NaN,True
9,9,4,57000.0,45000.0,False


### Operators and multiple columns

In the previous examples, the conditions involved only a single columns. However, it is possible to select multiple columns:

```
column_1 + column_2 + ... + column_n
```

Multiple column names can be provided by concatenating them with the plus sign (`+`).

```
column_*
```

Using the wildcard sign (`*`) selects all columns that start with `column_`. When multiple columns are selected, an `Operator` is needed to aggregate them to a single one. There are two types of `Operator`s:

#### DataOperator
A `DataOperator` performs an aggregation before it is send to a `Comparator`; it operates on the raw data. Common examples are `mean` or `sum`, which compute the mean and sum of the selected columns respectively.

#### LogicalOperators
A `LogicalOperator` performs an aggregation after a `Comparator` is used; it aggregates the boolean output from the `Comparator`. A common example is the `any` operator, which returns `True` if any of the input columns equals `True`.

Let's first start with a simple DataOperator; the example below shows how we can sum the income columns and then test it against some value.

In [8]:
# Check combined income is above 70.000
ps = Parser("sum income_* > 70000", "combined_income_above_70K")
households.join(ps.evaluate(households))

,id,size,income_1,income_2,combined_income_above_70K
0,0,1,35000.0,NaN,False
1,1,1,55000.0,NaN,False
2,2,1,110000.0,NaN,True
3,3,1,NaN,NaN,False
4,4,2,40000.0,35000.0,True
5,5,2,42000.0,22000.0,False
6,6,2,73000.0,NaN,True
7,7,2,38000.0,30000.0,False
8,8,3,NaN,NaN,False
9,9,4,57000.0,45000.0,True


What happens under the hood, is that `validata` first computes the sum of the `income_1` and `income_2` columns and then compares it to the value `70000`.

With logical operators the process is slightly different, but the idea is similar. Let's look at a simple example:

In [9]:
# Check any income is missing
ps = Parser("any income_* > 50000", "any_income_above_50K")
households.join(ps.evaluate(households))

,id,size,income_1,income_2,any_income_above_50K
0,0,1,35000.0,NaN,False
1,1,1,55000.0,NaN,True
2,2,1,110000.0,NaN,True
3,3,1,NaN,NaN,False
4,4,2,40000.0,35000.0,False
5,5,2,42000.0,22000.0,False
6,6,2,73000.0,NaN,True
7,7,2,38000.0,30000.0,False
8,8,3,NaN,NaN,False
9,9,4,57000.0,45000.0,True


Here `validata` first computes checks whether `income_1` and `income_2` are above `50000` which results in two boolean (`True` / `False`) values. After doing so, it combines the values using an `or` statement; if `any` of them is `True` the result is also `True`.

Note that you can embed operators in more complex statements, like so:

In [10]:
ps = Parser(
    """
    (size == 1 and all income_* missing) or
    (size > 1 and any income_* missing)
    """,
    "missing_income")
households.join(ps.evaluate(households))

,id,size,income_1,income_2,missing_income
0,0,1,35000.0,NaN,False
1,1,1,55000.0,NaN,False
2,2,1,110000.0,NaN,False
3,3,1,NaN,NaN,True
4,4,2,40000.0,35000.0,False
5,5,2,42000.0,22000.0,False
6,6,2,73000.0,NaN,True
7,7,2,38000.0,30000.0,False
8,8,3,NaN,NaN,True
9,9,4,57000.0,45000.0,False


To see which operators are available to you, use the `list()` method from the `Operator` base class, like so:

In [11]:
# List all available Operators
Operator.list()

{'all', 'any', 'max', 'mean', 'median', 'min', 'none', 'sum'}

## Validator: Performing many checks

### Define a data frame with validations

In [12]:
checks_df = pd.DataFrame(
    data=[
        ["large_size", "size > 2"],
        ["income_missing", "any income_* missing"],
        ["high_collective_income", "sum income_* > 100000"]
    ],
    columns=["name", "expression"]
)
checks_df

,name,expression
0,large_size,size > 2
1,income_missing,any income_* missing
2,high_collective_income,sum income_* > 100000


### Run all validations

In [13]:
vd = Validator(checks_df)
results = vd.validate(households)

In [14]:
results

,large_size,income_missing,high_collective_income
0,False,True,False
1,False,True,False
2,False,True,True
3,False,True,False
4,False,False,False
5,False,False,False
6,False,True,False
7,False,False,False
8,True,True,False
9,True,False,True


In [15]:
vd.get_summary()

,True %
large_size,20.0
income_missing,60.0
high_collective_income,20.0
